In [ ]:
import pandas as pd               # for data manipulation
import numpy as np                # for aggregation functions
import gc                         # for clean-up memory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Data_rec_sys

/content/drive/MyDrive/Data_rec_sys


Now we load all the orders (prior, train, test) with their metadata (order hour/day etc.).
Furthermore, we load the order_products_prior which includes all the products that have been purchased in each prior order.

In [ ]:
!unzip 'aisles.csv.zip' -d '/content/data'
!unzip 'departments.csv.zip' -d '/content/data'
!unzip 'order_products__prior.csv.zip' -d '/content/data'
!unzip 'order_products__train.csv.zip' -d '/content/data'
!unzip 'orders.csv.zip' -d '/content/data'
!unzip 'products.csv.zip' -d '/content/data'

Archive:  aisles.csv.zip
  inflating: /content/data/aisles.csv  
   creating: /content/data/__MACOSX/
  inflating: /content/data/__MACOSX/._aisles.csv  
Archive:  departments.csv.zip
  inflating: /content/data/departments.csv  
  inflating: /content/data/__MACOSX/._departments.csv  
Archive:  order_products__prior.csv.zip
  inflating: /content/data/order_products__prior.csv  
  inflating: /content/data/__MACOSX/._order_products__prior.csv  
Archive:  order_products__train.csv.zip
  inflating: /content/data/order_products__train.csv  
  inflating: /content/data/__MACOSX/._order_products__train.csv  
Archive:  orders.csv.zip
  inflating: /content/data/orders.csv  
  inflating: /content/data/__MACOSX/._orders.csv  
Archive:  products.csv.zip
  inflating: /content/data/products.csv  
  inflating: /content/data/__MACOSX/._products.csv  


In [ ]:
orders = pd.read_csv('/content/data/orders.csv')
order_products_prior = pd.read_csv('/content/data/order_products__prior.csv')

# Get the product tables also --> To create metrics for aisles / departments
products = pd.read_csv('/content/data/products.csv')
products.product_name = products.product_name.str.replace(' ', '_').str.lower()
products.product_name = products.product_name.str.replace(',', '-').str.lower()

In [ ]:
prd = orders.merge(order_products_prior, on='order_id', how='left')
prd.head()

#prd['product_name'] = products.product_name.str.replace(' ', '_').str.lower()
#prd['product_name'] = products.product_name.str.replace(',', '-').str.lower()
#TRIM DATASETS
#prd = prd.iloc[0:100000]
#prd = prd.iloc[0:15434766]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196.0,1.0,0.0
1,2539329,1,prior,1,2,8,NaN,14084.0,2.0,0.0
2,2539329,1,prior,1,2,8,NaN,12427.0,3.0,0.0
3,2539329,1,prior,1,2,8,NaN,26088.0,4.0,0.0
4,2539329,1,prior,1,2,8,NaN,26405.0,5.0,0.0


In [ ]:
uxp = prd.groupby(['user_id', 'product_id'])[['order_id']].count()
uxp.columns = ['uxp_total_bought']
uxp = uxp.reset_index()

In [ ]:
uxp.head()

,user_id,product_id,uxp_total_bought
0,1,196.0,10
1,1,10258.0,9
2,1,10326.0,1
3,1,12427.0,10
4,1,13032.0,3


In [ ]:
#Does users frequently reorder a product? (one-shot ratio)
item_one = uxp[uxp.uxp_total_bought==1].groupby('product_id')[['uxp_total_bought']].count()
item_one.columns = ['uxp_customers_one_shot']
item_size = uxp.groupby('product_id')[['user_id']].count()
item_size.columns = ['uxp_unique_customers']

userxproduct_var= item_one.merge(item_size, how='left', left_index=True, right_on='product_id')
userxproduct_var['one_shot_ratio_product'] = userxproduct_var.uxp_customers_one_shot / userxproduct_var.uxp_unique_customers
userxproduct_var = userxproduct_var.reset_index()


###########
uxp = uxp.merge(userxproduct_var[['product_id', 'one_shot_ratio_product']],how='left')
gc.collect()

13

In [ ]:
userxproduct_var

,product_id,uxp_customers_one_shot,uxp_unique_customers,one_shot_ratio_product
0,1.0,440,716,0.614525
1,2.0,70,78,0.897436
2,3.0,38,74,0.513514
3,4.0,118,182,0.648352
4,5.0,2,6,0.333333
...,...,...,...,...
49582,49684.0,7,8,0.875000
49583,49685.0,37,43,0.860465
49584,49686.0,20,36,0.555556
49585,49687.0,3,7,0.428571


In [ ]:
uxp.head()

,user_id,product_id,uxp_total_bought,one_shot_ratio_product
0,1,196.0,10,0.417500
1,1,10258.0,9,0.447038
2,1,10326.0,1,0.478419
3,1,12427.0,10,0.470518
4,1,13032.0,3,0.520218


In [ ]:
#last 5 orders
prd['order_number_back'] = prd.groupby('user_id')['order_number'].transform(max) - prd.order_number +1
prd5 = prd[prd.order_number_back <= 5]
last_five = prd5.groupby(['user_id','product_id'])[['order_id']].count()
last_five.columns = ['times_last5']
last_five['times_last5_ratio'] = last_five.times_last5 / 5

#############
uxp = uxp.merge(last_five , on=['user_id', 'product_id'], how='left')
del [last_five, prd5]
gc.collect()

<ipython-input-12-76e8972802a4>:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  prd['order_number_back'] = prd.groupby('user_id')['order_number'].transform(max) - prd.order_number +1


18

In [ ]:
uxp.head()

,user_id,product_id,uxp_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio
0,1,196.0,10,0.417500,4.0,0.8
1,1,10258.0,9,0.447038,4.0,0.8
2,1,10326.0,1,0.478419,NaN,NaN
3,1,12427.0,10,0.470518,4.0,0.8
4,1,13032.0,3,0.520218,2.0,0.4


In [ ]:
#How frequently a customer bought a product after its first purchase ?
times = prd.groupby(['user_id', 'product_id'])[['order_id']].count()
times.columns = ['Times_Bought_N']
total_orders = prd.groupby('user_id')[['order_number']].max()
total_orders.columns = ['total_orders']
first_order_number = prd.groupby(['user_id', 'product_id'])[['order_number']].min()
first_order_number.columns = ['first_order_number']
first_order_number_reset = first_order_number.reset_index()
span = pd.merge(total_orders, first_order_number_reset, on='user_id', how='right')
span['Order_Range_D'] = span.total_orders - span.first_order_number + 1
order_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
order_ratio['Order_Ratio_user_id_X_product_id'] = order_ratio.Times_Bought_N / order_ratio.Order_Range_D
del [times, total_orders, first_order_number, span]

###########
uxp = uxp.merge(order_ratio , on=['user_id', 'product_id'], how='left')
del order_ratio
gc.collect()

0

In [ ]:
uxp.head()

,user_id,product_id,uxp_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,Order_Ratio_user_id_X_product_id
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,0.909091
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,0.900000
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,0.142857
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,0.909091
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,0.300000


In [ ]:
#Which products have the highest probability of being reordered?
product_var = prd.groupby('product_id')[['reordered']].mean()
product_var.columns = ['reorder_ratio']

#What is the average position of a product in an order?
product_var['mean_add_to_cart_order'] = prd.groupby('product_id')[['add_to_cart_order']].mean()
product_var.head()

##########
uxp = uxp.merge(product_var, on='product_id', how='left')
del product_var
gc.collect()

0

In [ ]:
uxp.head()

,user_id,product_id,uxp_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,Order_Ratio_user_id_X_product_id,reorder_ratio,mean_add_to_cart_order
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,0.909091,0.776480,3.721774
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,0.900000,0.713772,4.277492
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,0.142857,0.652009,4.191097
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,0.909091,0.740735,4.760037
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,0.300000,0.657158,5.622767


In [ ]:
prd['user_max_onb'] = prd.groupby('user_id').order_number.transform(np.max)

from collections import defaultdict
item_cnt    = defaultdict(int)
item_chance = defaultdict(int)

<ipython-input-20-9a3b31d86407>:1: FutureWarning: The provided callable <function max at 0x7ec8df3e4ca0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  prd['user_max_onb'] = prd.groupby('user_id').order_number.transform(np.max)


In [ ]:
prd

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back,user_max_onb
0,2539329,1,prior,1,2,8,NaN,196.0,1.0,0.0,11,11
1,2539329,1,prior,1,2,8,NaN,14084.0,2.0,0.0,11,11
2,2539329,1,prior,1,2,8,NaN,12427.0,3.0,0.0,11,11
3,2539329,1,prior,1,2,8,NaN,26088.0,4.0,0.0,11,11
4,2539329,1,prior,1,2,8,NaN,26405.0,5.0,0.0,11,11
...,...,...,...,...,...,...,...,...,...,...,...,...
32640693,2977660,206209,prior,13,1,12,7.0,38730.0,6.0,0.0,2,14
32640694,2977660,206209,prior,13,1,12,7.0,31477.0,7.0,0.0,2,14
32640695,2977660,206209,prior,13,1,12,7.0,6567.0,8.0,0.0,2,14
32640696,2977660,206209,prior,13,1,12,7.0,22920.0,9.0,0.0,2,14


In [ ]:
pid_back = uid_back = onb_back = None

for user_id, product_id, order_number, max_onb in prd[['user_id', 'product_id', 'order_number', 'user_max_onb']].values:

    if user_id==uid_back and product_id==pid_back and (order_number-onb_back==1):
        item_cnt[product_id] +=1
    if order_number!=max_onb:
        item_chance[product_id] +=1

    uid_back = user_id
    pid_back = product_id
    onb_back = order_number

item_cnt = pd.DataFrame.from_dict(item_cnt, orient='index').reset_index()
item_cnt.columns = ['product_id', 'item_first_cnt']
item_chance = pd.DataFrame.from_dict(item_chance, orient='index').reset_index()
item_chance.columns = ['product_id', 'item_first_chance']
df = pd.merge(item_cnt, item_chance, on='product_id', how='outer').fillna(0)
df['item_first_ratio'] = df.item_first_cnt/df.item_first_chance

In [ ]:
df

,product_id,item_first_cnt,item_first_chance,item_first_ratio
0,1.0,20.0,1852,0.010799
1,2.0,0.0,90,0.000000
2,3.0,1.0,277,0.003610
3,4.0,0.0,329,0.000000
4,5.0,0.0,15,0.000000
...,...,...,...,...
49672,49684.0,0.0,9,0.000000
49673,49685.0,0.0,49,0.000000
49674,49686.0,1.0,120,0.008333
49675,49687.0,0.0,13,0.000000


In [ ]:
item_N2_cnt    = defaultdict(int)
item_N2_chance = defaultdict(int)
item_N3_cnt    = defaultdict(int)
item_N3_chance = defaultdict(int)
item_N4_cnt    = defaultdict(int)
item_N4_chance = defaultdict(int)
item_N5_cnt    = defaultdict(int)
item_N5_chance = defaultdict(int)

pid_back = uid_back = onb_back = None

for product_id, user_id, order_number, max_order_number in prd[['product_id', 'user_id', 'order_number','user_max_onb']].values:

    if product_id==pid_back and user_id==uid_back and (order_number-onb_back)<=2 and (max_order_number-order_number) >=2:
        item_N2_cnt[product_id] +=1
    if product_id==pid_back and user_id==uid_back and (max_order_number-order_number) >=2:
        item_N2_chance[product_id] +=1

    if product_id==pid_back and user_id==uid_back and (order_number-onb_back)<=3 and (max_order_number-order_number) >=3:
        item_N3_cnt[product_id] +=1
    if product_id==pid_back and user_id==uid_back and (max_order_number-order_number) >=3:
        item_N3_chance[product_id] +=1

    if product_id==pid_back and user_id==uid_back and (order_number-onb_back)<=4 and (max_order_number-order_number) >=4:
        item_N4_cnt[product_id] +=1
    if product_id==pid_back and user_id==uid_back and (max_order_number-order_number) >=4:
        item_N4_chance[product_id] +=1

    if product_id==pid_back and user_id==uid_back and (order_number-onb_back)<=5 and (max_order_number-order_number) >=5:
        item_N5_cnt[product_id] +=1
    if product_id==pid_back and user_id==uid_back and (max_order_number-order_number) >=5:
        item_N5_chance[product_id] +=1

    pid_back = product_id
    uid_back = user_id
    onb_back = order_number

In [ ]:
item_N2_cnt = pd.DataFrame.from_dict(item_N2_cnt, orient='index').reset_index()
item_N2_cnt.columns = ['product_id', 'item_N2_cnt']
item_N2_chance = pd.DataFrame.from_dict(item_N2_chance, orient='index').reset_index()
item_N2_chance.columns = ['product_id', 'item_N2_chance']

item_N3_cnt = pd.DataFrame.from_dict(item_N3_cnt, orient='index').reset_index()
item_N3_cnt.columns = ['product_id', 'item_N3_cnt']
item_N3_chance = pd.DataFrame.from_dict(item_N3_chance, orient='index').reset_index()
item_N3_chance.columns = ['product_id', 'item_N3_chance']

item_N4_cnt = pd.DataFrame.from_dict(item_N4_cnt, orient='index').reset_index()
item_N4_cnt.columns = ['product_id', 'item_N4_cnt']
item_N4_chance = pd.DataFrame.from_dict(item_N4_chance, orient='index').reset_index()
item_N4_chance.columns = ['product_id', 'item_N4_chance']

item_N5_cnt = pd.DataFrame.from_dict(item_N5_cnt, orient='index').reset_index()
item_N5_cnt.columns = ['product_id', 'item_N5_cnt']
item_N5_chance = pd.DataFrame.from_dict(item_N5_chance, orient='index').reset_index()
item_N5_chance.columns = ['product_id', 'item_N5_chance']


df2 = pd.merge(item_N2_cnt, item_N2_chance, on='product_id', how='outer')
df3 = pd.merge(item_N3_cnt, item_N3_chance, on='product_id', how='outer')
df4 = pd.merge(item_N4_cnt, item_N4_chance, on='product_id', how='outer')
df5 = pd.merge(item_N5_cnt, item_N5_chance, on='product_id', how='outer')

df_2_3_4_5 = pd.merge(pd.merge(df2, df3, on='product_id', how='outer'),
              pd.merge(df4, df5, on='product_id', how='outer'),
              on='product_id', how='outer').fillna(0)

df = df.merge(df_2_3_4_5, on='product_id', how='left')

df['item_N2_ratio'] = df['item_N2_cnt']/df['item_N2_chance']
df['item_N3_ratio'] = df['item_N3_cnt']/df['item_N3_chance']
df['item_N4_ratio'] = df['item_N4_cnt']/df['item_N4_chance']
df['item_N5_ratio'] = df['item_N5_cnt']/df['item_N5_chance']

In [ ]:
df.fillna(0, inplace=True)
df.reset_index(drop=True, inplace=True)

df=df[['product_id', 'item_first_ratio', 'item_N2_ratio','item_N3_ratio', 'item_N4_ratio', 'item_N5_ratio' ]]

del [item_cnt, item_chance, item_N2_cnt, item_N2_chance ,item_N3_cnt ,item_N3_chance,  item_N4_cnt  ,item_N4_chance,  item_N5_cnt, item_N5_chance, df_2_3_4_5]
gc.collect()


############
uxp = uxp.merge(df, on='product_id', how='left')
del df
gc.collect()

0

In [ ]:
#products['product_id'].unique()

array([    1,     2,     3, ..., 49686, 49687, 49688])

In [ ]:
total_products = 49688

In [ ]:
#Which aisle has the most products?
aisle_top = products.groupby('aisle_id')[['product_id']].count()
aisle_top.columns = ['total_products_aisle']
aisle_top.head()

prod_temp = products.merge(aisle_top, on='aisle_id', how='left')

dept_top = products.groupby('department_id')[['product_id']].count()
dept_top.columns = ['total_products_dept']
dept_top.head()

prod_temp = prod_temp.merge(dept_top, on='department_id', how='left')

prod_temp['total_products_aisle_ratio'] = prod_temp.total_products_aisle/total_products
prod_temp['total_products_dept_ratio'] = prod_temp.total_products_dept/total_products


#########
uxp = uxp.merge(prod_temp.drop(['aisle_id', 'department_id', 'product_name'],axis=1), on='product_id', how='left')
del prod_temp, products
gc.collect()

NameError: name 'total_products' is not defined

In [ ]:
#ass 3
#Get the average, maximum & minimum order size for each customer.
order_size = prd.groupby(['user_id', 'order_id'])[['product_id']].count()
order_size.columns = ['size']
results = order_size.groupby('user_id')[['size']].mean()
results.columns = ['order_size_avg']
results = results.reset_index()

##########
uxp = uxp.merge(results, on=['user_id'], how='left')
del [order_size, results]
gc.collect()

49

In [ ]:
uxp

,user_id,product_id,uxp_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,Order_Ratio_user_id_X_product_id,reorder_ratio,mean_add_to_cart_order,order_size_avg
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,0.909091,0.776480,3.721774,5.363636
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,0.900000,0.713772,4.277492,5.363636
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,0.142857,0.652009,4.191097,5.363636
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,0.909091,0.740735,4.760037,5.363636
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,0.300000,0.657158,5.622767,5.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,43961.0,3,0.485358,1.0,0.2,3,14,4,11,0.272727,0.630583,9.194723,9.214286
13307949,206209,44325.0,1,0.710110,NaN,NaN,1,14,7,8,0.125000,0.401148,10.109900,9.214286
13307950,206209,48370.0,1,0.407439,1.0,0.2,1,14,11,4,0.250000,0.699288,8.344942,9.214286
13307951,206209,48697.0,1,0.739338,NaN,NaN,1,14,7,8,0.125000,0.357661,8.763058,9.214286


In [ ]:
'''#Find the size (number of products) of each order. (1)
order_var = order_products.groupby('order_id')[['product_id']].count()
order_var.columns= ['order_size']
order_var = order_var.reset_index()

#Find the number of orders for each basket size. (2)
size_results = order_var.groupby('order_size')[['order_size']].count()
size_results.columns = ['total_orders']

# merge (2) to (1)
order_var = order_var.merge(size_results, on='order_size', how='left')
order_var.head()

# How frequent an order has reordered products?
order_var = order_var.set_index('order_id')

order_var['reordered_ratio_order']=order_products.groupby('order_id')[['reordered']].mean()
order_var.head()

order_var = order_var.reset_index()

prd = prd.merge(order_var[['order_id', 'reordered_ratio_order']], on='order_id', how='left')

del [order_var , size_results]
gc.collect()'''

"#Find the size (number of products) of each order. (1)\norder_var = order_products.groupby('order_id')[['product_id']].count()\norder_var.columns= ['order_size']\norder_var = order_var.reset_index()\n\n#Find the number of orders for each basket size. (2)\nsize_results = order_var.groupby('order_size')[['order_size']].count()\nsize_results.columns = ['total_orders']\n\n# merge (2) to (1)\norder_var = order_var.merge(size_results, on='order_size', how='left')\norder_var.head()\n\n# How frequent an order has reordered products?\norder_var = order_var.set_index('order_id')\n               \norder_var['reordered_ratio_order']=order_products.groupby('order_id')[['reordered']].mean()\norder_var.head()\n\norder_var = order_var.reset_index()\n\nprd = prd.merge(order_var[['order_id', 'reordered_ratio_order']], on='order_id', how='left')\n\ndel [order_var , size_results]\ngc.collect()"

In [ ]:
uxp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 14 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   user_id                           int64  
 1   product_id                        float64
 2   uxp_total_bought                  int64  
 3   one_shot_ratio_product            float64
 4   times_last5                       float64
 5   times_last5_ratio                 float64
 6   Times_Bought_N                    int64  
 7   total_orders                      int64  
 8   first_order_number                int64  
 9   Order_Range_D                     int64  
 10  Order_Ratio_user_id_X_product_id  float64
 11  reorder_ratio                     float64
 12  mean_add_to_cart_order            float64
 13  order_size_avg                    float64
dtypes: float64(8), int64(6)
memory usage: 1.4 GB


In [ ]:
uxp.head(30)

,user_id,product_id,uxp_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,Order_Ratio_user_id_X_product_id,reorder_ratio,mean_add_to_cart_order,order_size_avg
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,0.909091,0.776480,3.721774,5.363636
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,0.900000,0.713772,4.277492,5.363636
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,0.142857,0.652009,4.191097,5.363636
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,0.909091,0.740735,4.760037,5.363636
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,0.300000,0.657158,5.622767,5.363636
5,1,13176.0,2,0.288116,NaN,NaN,2,11,2,10,0.200000,0.832555,5.095947,5.363636
6,1,14084.0,1,0.370850,NaN,NaN,1,11,1,11,0.090909,0.810982,5.792595,5.363636
7,1,17122.0,1,0.472574,NaN,NaN,1,11,5,7,0.142857,0.675576,6.257421,5.363636
8,1,25133.0,8,0.419255,4.0,0.8,8,11,3,9,0.888889,0.740155,7.001614,5.363636
9,1,26088.0,2,0.595013,NaN,NaN,2,11,1,11,0.181818,0.539041,6.495838,5.363636


In [ ]:
uxp.to_pickle('uxp.pkl')
uxp.to_csv('uxp.csv')